In [ ]:
import numpy as np
import pandas as pd
import pickle

In [3]:
def find_similar_movies(items_emb_final, movie_index, top_k=10):
    # 获取目标电影的嵌入
    target_emb = items_emb_final[movie_index]  # shape: (embedding_dim,)
    
    # 计算余弦相似度
    norm_items_emb = items_emb_final / np.linalg.norm(items_emb_final, axis=1, keepdims=True)  # 单位化每个电影的嵌入
    norm_target_emb = target_emb / np.linalg.norm(target_emb)  # 单位化目标电影的嵌入
    similarities = np.dot(norm_items_emb, norm_target_emb)  # shape: (num_movies,)
    
    # 排除目标电影本身
    similarities[movie_index] = -1.0  # 设置自身的相似度为 -1，避免被选中
    
    # 获取 Top-K 相似电影
    topk_indices = np.argsort(similarities)[-top_k:][::-1]  # 从大到小排序并取前K个索引
    
    return topk_indices.tolist()

In [5]:
# 加载时使用以下代码
# 加载 embeddings
dataset = 'ml-25m-reduced'
users_embeddings_file = './save/'+dataset+'/users_embeddings.npy'
items_embeddings_file = './save/'+dataset+'/items_embeddings.npy'
maps_file = './save/'+dataset+'/dataset_maps.pkl'
users_embeddings = np.load(users_embeddings_file)
items_embeddings = np.load(items_embeddings_file)

# 加载 maps
with open(maps_file, 'rb') as f:
    loaded_maps = pickle.load(f)

movie_map = loaded_maps['movie_map']
user_map = loaded_maps['user_map']

print("Embeddings and maps loaded successfully.")

# 为指定电影推荐相似电影
movies_df = pd.read_csv("../data/"+dataset+"/movies.csv")
movie_index  = 10
similar_indices = find_similar_movies(items_embeddings, movie_index, top_k=10)

movie_mapping_inverse = {v: k for k, v in movie_map.items()}
target_movie_name = movies_df[movies_df['movieId'] == movie_mapping_inverse[movie_index]]['title'].values[0]
similar_ids = [movie_mapping_inverse[indice] for indice in similar_indices]
filtered_df = movies_df[movies_df['movieId'].isin(similar_ids)]
similar_movie_names = filtered_df['title'].tolist()

print(f"Similar movies to '{target_movie_name}':")
print(similar_movie_names)


Embeddings and maps loaded successfully.
Similar movies to 'American President, The (1995)':
["Mr. Holland's Opus (1995)", 'French Kiss (1995)', 'While You Were Sleeping (1995)', 'Four Weddings and a Funeral (1994)', 'Dave (1993)', 'Sleepless in Seattle (1993)', 'Witness (1985)', "You've Got Mail (1998)", 'League of Their Own, A (1992)', 'Erin Brockovich (2000)']
